In [14]:
import pandas as pd
import openpyxl

In [15]:
# Подгружаем данные с рабочего стола)

In [16]:
df = pd.read_excel('/Users/mars/Desktop/Задание Dash copy.xlsx')

In [17]:
df

,дата,продукт,сумма
0,2023-01-01,A,100
1,2023-01-01,B,200
2,2023-01-01,C,50
3,2023-02-01,A,120
4,2023-02-01,B,70
5,2023-02-01,C,0
6,2023-03-01,A,80
7,2023-03-01,B,30
8,2023-03-01,C,50
9,2023-04-01,A,150


In [18]:
report_table = df.pivot_table(index='дата',
                                      columns='продукт',
                                      values='сумма',
                                      aggfunc='sum').round(0)

In [19]:
report_table

продукт,A,B,C,D
дата,,,,
2023-01-01,100.0,200.0,50.0,NaN
2023-02-01,120.0,70.0,0.0,NaN
2023-03-01,80.0,30.0,50.0,NaN
2023-04-01,150.0,120.0,30.0,NaN
2023-05-01,100.0,80.0,50.0,90.0


In [20]:
new_line = {'A': report_table['A'].sum(), 'B': report_table['B'].sum(), 'C':report_table['C'].sum(), 'D':report_table['D'].sum() }

In [21]:
rp = report_table.append(new_line, ignore_index = True)

/var/folders/gk/9tn7jfr543q1zx_jmhprn1jw0000gn/T/ipykernel_91522/2219337538.py:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [22]:
rp

,A,B,C,D
0,100.0,200.0,50.0,NaN
1,120.0,70.0,0.0,NaN
2,80.0,30.0,50.0,NaN
3,150.0,120.0,30.0,NaN
4,100.0,80.0,50.0,90.0
5,550.0,500.0,180.0,90.0


In [23]:
rp.set_axis(['янв', 'фев', 'мар', 'апр', 'май', 'Общий итог'], axis = 0, inplace = True )

In [24]:
rp

,A,B,C,D
янв,100.0,200.0,50.0,NaN
фев,120.0,70.0,0.0,NaN
мар,80.0,30.0,50.0,NaN
апр,150.0,120.0,30.0,NaN
май,100.0,80.0,50.0,90.0
Общий итог,550.0,500.0,180.0,90.0


In [25]:
# Импорт необходимых библиотек
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px

#обращаемся к заранее подготовленным данным
df = rp

# инициализация
app = Dash(__name__)

# макет
app.layout = html.Div([
    html.Div(children='Sales report'),
    html.Hr(),
    dcc.Checklist(options=[{'label': col, 'value': col} for col in df.columns], value=['A'], id='controls-and-checkbox-item'),
    dash_table.DataTable(data=df.to_dict('records'), columns=[{'name': col, 'id': col} for col in df.columns], page_size=6),
    dcc.Graph(figure={}, id='controls-and-graph')
])

#определение колбэка для обновления графика при изменении выбранных флагов
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-checkbox-item', component_property='value')
)
def update_graph(cols_chosen):
    #создание графика
    fig = px.bar(df, x=df.index, y=cols_chosen)
    
    # Заголовок и названия осей
    fig.update_layout(
        title=f'Продукты {", ".join(cols_chosen)}',
        xaxis_title='Месяц',
        yaxis_title=f'Сумма'
    )
    return fig

# Запуск
if __name__ == '__main__':
    app.run(debug=True)